# Dependencies

In [ ]:
import os
import cv2
import winsound
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt

# Constants

In [ ]:
DATA_PATH = os.path.join('D:\\DATA')
SEQUENCE_LENGTH = 30

# Driver Functions

In [ ]:
mp_holistic = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils #drawing utilities


def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #color conversion
    image.flags.writeable = False #image is no longer writable
    results = model.process(image) #make pred 
    image.flags.writeable = False #image is writable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #color conversion
    return image, results


def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                            )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                            )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                            )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                            )

    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])


def collectData(actionName, currentPath, cap):
    os.makedirs(currentPath)
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for i in range(SEQUENCE_LENGTH):
            ret, frame = cap.read() #Capture the video frame by frame
            image, results = mediapipe_detection(frame, holistic)  #make detection
                
            draw_styled_landmarks(image, results) #draw
        
            #apply logic
            if i == 0:
                cv2.putText(image, "STARTING COLLECTION", (120, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.waitKey(1000)
                winsound.Beep(440, 500)
                
            cv2.putText(image, "ACTION# {} FRAME# {}".format(actionName, i), (120, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                
            keypoints = extract_keypoints(results)
            newPath = os.path.join(currentPath, str(i))
            np.save(newPath, keypoints)
            cv2.imshow('DATA COLLECTION', image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                return
            
    print(currentPath)

# Action Input

In [ ]:
action = input("action:")

# Video Input

In [ ]:
start = int(input("start:"))
end = int(input("end:"))

cap = cv2.VideoCapture(0)
for i in range(start, end+1):
    try:
        currentPath = os.path.join(DATA_PATH, action, str(i))
        collectData(action, currentPath, cap)
    except:
        pass

cap.release()
cv2.destroyAllWindows()